In [1]:
# import collections 
# import sys
# if sys.version_info.major == 3 and sys.version_info.minor >= 10:

#     from collections.abc import MutableMapping
# else:
#     from collections import MutableMapping
from plot_functions import *
from global_file import create_params_dict
from connection_and_distillation import *
import time
from itertools import product
import warnings

warnings.filterwarnings("error")
warnings.filterwarnings(
    "ignore", message="elementwise comparison failed; returning scalar instead, "
                      "but in the future will perform elementwise comparison")


In [49]:
import networkx as nx
import numpy as np

def insert_chain(G, start_node, end_node, n):
    """
    Inserts a non-linear chain of n repeaters between start_node and end_node in the graph G.
    """
    for i in range(n):
        new_node = chr(ord('C') + i)  # Use consecutive letters for new nodes
        G.add_node(new_node, pos=np.array([0, 0]))  # Initialize positions (you can adjust this as needed)
        random_existing_node = np.random.choice(list(G.nodes()))  # Choose a random existing node
        G.add_edge(random_existing_node, new_node)  # Connect the new node to a random existing node
        G.add_edge(new_node, end_node)  # Connect the new node to the end_node
    return G

def create_repeater_network(distance, n, num_paths=1):
    """
    Returns a graph G that is a non-linear network of repeaters with multiple paths from A to B.
    """
    G = nx.Graph()
    G.add_node('A', pos=np.array([0, 0]))
    G.add_node('B', pos=np.array([0, distance]))

    for _ in range(num_paths):
        G = insert_chain(G, 'A', 'B', n)

    perturbation = global_file.params.pert  # Assuming pert is defined in global_file.py
    if perturbation:
        # Add random displacements to the node positions
        for node in G.nodes():
            G.nodes[node]['pos'] += np.random.uniform(-perturbation, perturbation, size=2)

    nx.set_edge_attributes(G, values=global_file.params.number_of_fibres, name='#fibres')
    return G

In [47]:
# Example usage
distance = 10  # Adjust as needed
num_repeaters = 10  # Number of repeaters in each path
num_paths = 3  # Number of paths from A to B
graph = create_repeater_network(distance, num_repeaters, num_paths)


In [39]:
filename = 'parameters/myfile.txt'

global_file.params = create_params_dict(filename)

G = create_repeater_network(global_file.params.distance, int(global_file.params.n),)
G = assign_parameters_to_nodes(G)



# with open('pkl_files/' + filename + '.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(path_dict, f)

# plot_fidelity_vs_time_plot(G, 'A', 'B', label=filename,
#                            color=None, connected=True)
# scheme1 = Scheme(('A'), None, None, None, [0.9, 0, 0, 0.1], 1, 1)
# generate_pdf_of_scheme_graph(scheme, "A", general_distillation=False)


In [34]:
candidate_paths=optimise_schemes(G, global_file.params.general_distillation,
                  symmetricoptimisation=global_file.params.symmetricoptimisation)

Number of pruned schemes is 48, from 57 to 9.
Finished elementary links.
Number of pruned schemes is 0, from 8 to 8.
Time for swap protocols =    0.84284
Time for dist protocols =    0.00004
--
25.0 %
Number of pruned schemes is 0, from 4 to 4.
Time for swap protocols =    0.65238
Time for dist protocols =    0.00014
--
33.33 %
Number of pruned schemes is 0, from 12 to 12.
Time for swap protocols =    0.95631
Time for dist protocols =    0.00004
--
41.67 %
Number of pruned schemes is 0, from 12 to 12.
Time for swap protocols =    1.17932
Time for dist protocols =    0.00009
--
50.0 %
Number of pruned schemes is 2, from 12 to 10.
Time for swap protocols =    2.02273
Time for dist protocols =    0.00003
--
58.33 %
Number of pruned schemes is 2, from 12 to 10.
Time for swap protocols =    1.81332
Time for dist protocols =    0.00003
--
66.67 %
Number of pruned schemes is 0, from 19 to 19.
Time for swap protocols =    2.98277
Time for dist protocols =    0.00004
--
75.0 %
Number of pruned 

In [35]:
candidate_paths

[('A',
  'AB1',
  'AB2',
  'AB3',
  'AB4',
  'AB5',
  'AB6',
  'AB7',
  'AB8',
  'AB9',
  'AB10',
  'B')]

In [51]:
pos = nx.spring_layout(G, k=0.9, seed=42)  # Adjust the value of k # Set a seed value for reproducibility
edges = G.edges()

nx.drawing.nx_agraph.write_dot(G, "network.dot")
# nx.draw(G, pos, with_labels=True, font_size=8)
import subprocess
subprocess.run(f"dot -Tsvg network.dot > network.svg", shell=True)

CompletedProcess(args='dot -Tsvg network.dot > network.svg', returncode=0)

In [17]:
import subprocess
subprocess.run(f"dot -Tsvg network.dot > network.svg", shell=True)

CompletedProcess(args='dot -Tsvg network.dot > network.svg', returncode=0)